### Script to download nyc open data on health violations

In [1]:
import os
import sys
import urllib
import numpy as np
import pandas as pd
#import geopandas as gpd
# import matplotlib.pyplot as plt
import psycopg2
from sqlalchemy import create_engine

sys.path.append('../')
from envir import config
#%matplotlib inline

In [2]:
engine = create_engine('postgresql+psycopg2:///yelp_abcs')
con = engine.connect()

In [3]:
! curl https://data.cityofnewyork.us/api/views/43nn-pn8j/rows.csv?accessType=DOWNLOAD > nycinspect.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  161M    0  161M    0     0  7865k      0 --:--:--  0:00:21 --:--:-- 7807k


In [4]:
nyc_inspect = pd.read_csv('nycinspect.csv')

os.system('mv nycinspect.csv '+ config.shared)

nyc_inspect.head()

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,...,RECORD DATE,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,41302692,ARMANDO'S POLLO RICO RESTAURANT,Queens,11002,JAMAICA AVENUE,11418.0,7184418441,"Latin (Cuban, Dominican, Puerto Rican, South &...",11/08/2018,Violations were cited in the following area(s).,...,11/15/2019,Cycle Inspection / Initial Inspection,40.696345,-73.837906,409.0,28.0,12601.0,4194813.0,4.093000e+09,QN54
1,50005854,TENDO SUSHI,Queens,6626,FRESH POND RD,11385.0,7183810809,Japanese,06/27/2016,Violations were cited in the following area(s).,...,11/15/2019,Cycle Inspection / Re-inspection,40.708195,-73.897878,405.0,30.0,58700.0,4084889.0,4.035280e+09,QN20
2,50015204,HOOTERS,Manhattan,155,W 33RD ST,10001.0,2126959580,American,12/11/2017,Violations were cited in the following area(s).,...,11/15/2019,Cycle Inspection / Initial Inspection,40.749768,-73.989772,105.0,3.0,10100.0,1015195.0,1.008090e+09,MN17
3,40812973,LUNELLA RISTORANTE,Manhattan,173,MULBERRY STREET,10013.0,2129666639,Italian,10/30/2019,Establishment Closed by DOHMH. Violations wer...,...,11/15/2019,Cycle Inspection / Initial Inspection,40.720158,-73.996980,102.0,1.0,4100.0,1006964.0,1.004710e+09,MN24
4,50000905,SUBWAY,Manhattan,2699,BROADWAY,10025.0,2129610632,Sandwiches/Salads/Mixed Buffet,10/17/2019,Violations were cited in the following area(s).,...,11/15/2019,Cycle Inspection / Re-inspection,40.799033,-73.968599,107.0,6.0,19100.0,1056548.0,1.018740e+09,MN12


In [5]:
# move to db
#nyc_inspect.to_sql('nyc_inspections_test_2', engine, schema='stag', index=False)

In [6]:
nyc_inspect.to_sql('nyc_inspections_test_2', engine, schema='clean', index=False) 
nyc_inspect.to_sql('nyc_inspections_test_2', engine, schema='explore', index=False) 
nyc_inspect.to_sql('nyc_inspections_test_2', engine, schema='model', index=False)

In [7]:
nyc_inspect.shape

(396199, 26)

In [8]:
# Number of unique restaurants

nyc_inspect['CAMIS'].nunique()

27092

In [9]:
nyc_inspect.columns

Index(['CAMIS', 'DBA', 'BORO', 'BUILDING', 'STREET', 'ZIPCODE', 'PHONE',
       'CUISINE DESCRIPTION', 'INSPECTION DATE', 'ACTION', 'VIOLATION CODE',
       'VIOLATION DESCRIPTION', 'CRITICAL FLAG', 'SCORE', 'GRADE',
       'GRADE DATE', 'RECORD DATE', 'INSPECTION TYPE', 'Latitude', 'Longitude',
       'Community Board', 'Council District', 'Census Tract', 'BIN', 'BBL',
       'NTA'],
      dtype='object')

In [10]:
nyc_inspect['INSPECTION DATE'].min()

'01/01/1900'

The date 01/01/1900 refers to restaurants that haven't been inspected yet.

In [11]:
nyc_inspect['INSPECTION DATE'].max()

'12/31/2018'

In [12]:
nyc_active = nyc_inspect[nyc_inspect['INSPECTION DATE'] > '01/01/1900'].copy()

In [13]:
nyc_active.shape

(394571, 26)

Removed about 2,000 rows from dataset. Now contains only restaurants that have been inspected.

In [14]:
nyc_active['INSPECTION DATE'].min()

'01/02/2016'

In [15]:
nyc_active['VIOLATION DESCRIPTION'].unique

<bound method Series.unique of 0         Facility not vermin proof. Harborage or condit...
1         Raw, cooked or prepared food is adulterated, c...
2         Non-food contact surface improperly constructe...
3         Plumbing not properly installed or maintained;...
4         Filth flies or food/refuse/sewage-associated (...
5         Cold food item held above 41º F (smoked fish a...
6         “Choking first aid” poster not posted. “Alcoho...
7         Raw, cooked or prepared food is adulterated, c...
8         Proper sanitization not provided for utensil w...
9         Proper sanitization not provided for utensil w...
10        Pesticide use not in accordance with label or ...
11        Pesticide use not in accordance with label or ...
12        Cold food item held above 41º F (smoked fish a...
14        Evidence of mice or live mice present in facil...
15        Cold food item held above 41º F (smoked fish a...
16        Cold food item held above 41º F (smoked fish a...
17       

In [16]:
nyc_active['VIOLATION CODE'].nunique()

106

In [17]:
nyc_active['VIOLATION DESCRIPTION'].nunique()

91

In [18]:
nyc_active['VIOLATION DESCRIPTION']

0         Facility not vermin proof. Harborage or condit...
1         Raw, cooked or prepared food is adulterated, c...
2         Non-food contact surface improperly constructe...
3         Plumbing not properly installed or maintained;...
4         Filth flies or food/refuse/sewage-associated (...
5         Cold food item held above 41º F (smoked fish a...
6         “Choking first aid” poster not posted. “Alcoho...
7         Raw, cooked or prepared food is adulterated, c...
8         Proper sanitization not provided for utensil w...
9         Proper sanitization not provided for utensil w...
10        Pesticide use not in accordance with label or ...
11        Pesticide use not in accordance with label or ...
12        Cold food item held above 41º F (smoked fish a...
14        Evidence of mice or live mice present in facil...
15        Cold food item held above 41º F (smoked fish a...
16        Cold food item held above 41º F (smoked fish a...
17        Single service item reused, im

In [19]:
nyc_violations = pd.DataFrame({'violation_code':nyc_active['VIOLATION CODE'], 
                               'violation_description':nyc_active['VIOLATION DESCRIPTION']})

In [20]:
nyc_violations.head()

,violation_code,violation_description
0,08A,Facility not vermin proof. Harborage or condit...
1,04H,"Raw, cooked or prepared food is adulterated, c..."
2,10F,Non-food contact surface improperly constructe...
3,10B,Plumbing not properly installed or maintained;...
4,04N,Filth flies or food/refuse/sewage-associated (...


In [21]:
nyc_violations.to_csv(config.shared + '/nyc_violations.csv')

In [22]:
#nyc_violations.to_sql('nyc_violations_test_2', engine, schema = 'stag', index = False)

In [23]:
nyc_violations.to_sql('nyc_violations_test_2', engine, schema = 'clean', index = False)
nyc_violations.to_sql('nyc_violations_test_2', engine, schema = 'explore', index = False)
nyc_violations.to_sql('nyc_violations_test_2', engine, schema = 'model', index = False)